# Official Demo

In [1]:
!pip install umbral

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 6.3 MB/s eta 0:00:00a 0:00:01


In [3]:
from umbral import SecretKey, Signer

# Generate Umbral keys for Alice.
alices_secret_key = SecretKey.random()
alices_public_key = alices_secret_key.public_key()

alices_signing_key = SecretKey.random()
alices_signer = Signer(alices_signing_key)
alices_verifying_key = alices_signing_key.public_key()

# Generate Umbral keys for Bob.
bobs_secret_key = SecretKey.random()
bobs_public_key = bobs_secret_key.public_key()


In [4]:
alices_secret_key

In [5]:
from umbral import encrypt, decrypt_original

# Encrypt data with Alice's public key.
plaintext = b'Proxy Re-Encryption is cool!'
capsule, ciphertext = encrypt(alices_public_key, plaintext)

# Decrypt data with Alice's private key.
cleartext = decrypt_original(alices_secret_key, capsule, ciphertext)

In [7]:
cleartext

b'Proxy Re-Encryption is cool!'

## Bob side

In [8]:
from umbral import generate_kfrags

kfrags = generate_kfrags(
    delegating_sk=alices_secret_key,   # Alice’s private key
    receiving_pk=bobs_public_key,      # Bob’s public key
    signer=alices_signer,              # Alice’s signer (for security & verification)
    threshold=10,                      # Minimum number of fragments required to decrypt
    shares=20                          # Total number of fragments created
)

## Re-encryption

In [9]:
from umbral import reencrypt, pre

# Several Ursulas perform re-encryption, and Bob collects the resulting `cfrags`.
cfrags = list()           # Bob's cfrag collection
for kfrag in kfrags[:10]:
    cfrag = pre.reencrypt(capsule=capsule, kfrag=kfrag)
    cfrags.append(cfrag)    # Bob collects a cfrag

In [11]:
cfrags

## Decryption by Bob

In [16]:
from umbral import decrypt_reencrypted

bob_cleartext = pre.decrypt_reencrypted(receiving_sk=bobs_secret_key,
                                        delegating_pk=alices_public_key,
                                        capsule=capsule,
                                        verified_cfrags=cfrags,
                                        ciphertext=ciphertext)
assert bob_cleartext == plaintext

In [17]:
bob_cleartext

b'Proxy Re-Encryption is cool!'

In [15]:
help(pre.decrypt_reencrypted)

Help on function decrypt_reencrypted in module umbral.pre:

decrypt_reencrypted(receiving_sk: umbral.keys.SecretKey, delegating_pk: umbral.keys.PublicKey, capsule: umbral.capsule.Capsule, verified_cfrags: Sequence[umbral.capsule_frag.VerifiedCapsuleFrag], ciphertext: bytes) -> bytes
    Decrypts the ciphertext using the original capsule and the reencrypted capsule fragments.

